![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F01+-+Data+Sources&dt=01+-+BigQuery+-+Table+Data+Source.ipynb)

# 01 - BigQuery - Table Data Source
Use BigQuery to load and prepare data for machine learning:

**Video Walkthrough of this notebook:**

Includes conversational walkthrough and more explanatory information than the notebook:
<p align="center" width="100%"><center><a href="https://youtu.be/Z5whg20WvS8" target="_blank" rel="noopener noreferrer"><img src="../architectures/thumbnails/playbutton/01.png" width="40%"></a></center></p>

**Prerequisites:**
-  [00 - Environment Setup](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb)

**Resources:**
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Download BigQuery Data to Pandas](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)
-  [BigQuery Template Notebooks](https://github.com/GoogleCloudPlatform/bigquery-notebooks/tree/main/notebooks/official/template_notebooks)
- Using BigQuery From Python, Notebooks in This Repository:
    - [Tips/BigQuery - Python Client](../Tips/BigQuery%20-%20Python%20Client.ipynb)
    - [03 - BigQuery ML (BQML)/Introduction to BigQuery ML (BQML)](../03%20-%20BigQuery%20ML%20(BQML)/Introduction%20to%20BigQuery%20ML%20(BQML).ipynb)
    - [Applied Forecasting/1 - BigQuery Time Series Forecasting Data Review and Preparation](../Applied%20Forecasting/1%20-%20BigQuery%20Time%20Series%20Forecasting%20Data%20Review%20and%20Preparation.ipynb)

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/01_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/01_console.png" width="45%">
</p>

---
## Source Data

**Overview**

This notebook imports source data for this project into Google BigQuery.  All the remaining notebooks utilize BigQuery as the source and leverage API's native to the machine learning approaches they feature.

In the enviornment setup notebook (00), a BigQuery source table was exported to CSV format in a Cloud Storage Bucket. This notebook, `01 - BigQuery - Table Data Source`, starts the machine learning lifecycle by importing source data and preparing it for machine learning.

All of these workflows utilize tabular data to fit a supervised learning model: predict a target variable by learning patterns in feature columns.  The type of supervised learning used in these projects is classification: models with a target variable that has multiple discrete classes.  

**The Data**

The source data is first exported to Google Cloud Storage in CSV format below.  The BigQuery source table is `bigquery-public-data.ml_datasets.ulb_fraud_detection`.  This is a table of credit card transactions that are classified as fradulant, `Class = 1`, or normal `Class = 0`.    
- The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).
- Read mode about BigQuery public datasets [here](https://cloud.google.com/bigquery/public-data)

**Description of the Data**

This is a table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.  In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction
>**Quick Note on PCA**<p>PCA is an unsupervised learning technique: there is not a target variable.  PCA is commonlly used as a variable/feature reduction technique.  If you have 100 features then you could reduce it to a number p (say 10) projected features.  The choice of this number is a balance of how well it can explain the variance of the full feature space and reducing the number of features.  Each projected feature is orthogonal to each other feature, meaning there is no correlation between these new projected features.</p>

**Preparation of the Data**

This notebook adds two columns to the source data and stores it in a new table with suffix `_prepped`.  
- `transaction_id` (string) a unique id for the row/transaction
- `splits` (string) this divided the tranactions into sets for `TRAIN` (80%), `VALIDATE` (10%), and `TEST` (10%)

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'onboarding-host-5c79d5c5a45a40'

In [2]:
REGION = 'us-west1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

packages:

In [3]:
from google.cloud import bigquery
from google.cloud import storage

clients:

In [4]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

parameters:

In [5]:
BUCKET = PROJECT_ID

---
## Store the Source Data in GCS Storage Bucket
Check to see if the export exist and create if not:
- export from bigquery table to GCS bucket as CSV
    - the table is referenced in the `BQ_SOURCE` variable at the top of this notebook
- [Exporting Table Data](https://cloud.google.com/bigquery/docs/exporting-data#python)
- [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_extract_table)

For more tips on interacting with GCS using the Python Client review the tips notebook [Python Client for GCS](../Tips/Python%20Client%20for%20GCS.ipynb).

In [6]:
file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"

In [7]:
bucketDef = gcs.bucket(BUCKET)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Creating the export ...')
    extract.result()
    print(f'Exported the table to: gs://{bucketDef.name}/{file}')

The file has already been created at: gs://onboarding-host-5c79d5c5a45a40/01/01/data/fraud.csv


list files in the bucket:

In [8]:
list(bucketDef.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}'))

[<Blob: onboarding-host-5c79d5c5a45a40, 01/01/data/fraud.csv, 1683755005427471>]

In [9]:
print(f'Review the files in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')

Review the files in the console here:
https://console.cloud.google.com/storage/browser/onboarding-host-5c79d5c5a45a40/01;tab=objects&project=onboarding-host-5c79d5c5a45a40


---
## Create BigQuery Dataset

List BigQuery datasets in the project:

In [10]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud


Create the dataset if missing:

In [11]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

List BigQuery datasets in the project:

In [12]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud


---
## Create BigQuery Table
- import data from Cloud Storage Bucket
- [Loading CSV data from Cloud Storage](https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv)
- [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_extract_table)

In [13]:
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

The table already exists: onboarding-host-5c79d5c5a45a40.fraud.fraud


### Retrieve and Review a Sample From The Table:
> **Note:** The `LIMIT 5` statement does limit the number of rows returned by BigQuery to 5 but BigQuery still does a full table scan.  If you have a table larger than 1GB and want to limit the rows scanned for a quick review like then then replacing `LIMIT 5` with `TABLESAMPLE SYSTEM (1 PERCENT)` would be more efficient.  For tables under 1GB it will still return the full table.  More on [Table Sampling](https://cloud.google.com/bigquery/docs/table-sampling)

In [14]:
!pip3 install pandas==1.2.4 --upgrade --user

In [15]:
!pip3 install db-dtypes

In [16]:
import db_dtypes
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.00,0
1,27074,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.00,0
2,28292,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.00,0
3,28488,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.00,0
4,31392,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,154599,0.667714,3.041502,-5.845112,5.967587,0.213863,-1.462923,-2.688761,0.677764,-3.447596,...,0.329760,-0.941383,-0.006075,-0.958925,0.239298,-0.067356,0.821048,0.426175,6.74,1
284803,90676,-2.405580,3.738235,-2.317843,1.367442,0.394001,1.919938,-3.106942,-10.764403,3.353525,...,10.005998,-2.454964,1.684957,0.118263,-1.531380,-0.695308,-0.152502,-0.138866,6.99,1
284804,34634,0.333499,1.699873,-2.596561,3.643945,-0.585068,-0.654659,-2.275789,0.675229,-2.042416,...,0.469212,-0.144363,-0.317981,-0.769644,0.807855,0.228164,0.551002,0.305473,18.96,1
284805,96135,-1.952933,3.541385,-1.310561,5.955664,-1.003993,0.983049,-4.587235,-4.892184,-2.516752,...,-1.998091,1.133706,-0.041461,-0.215379,-0.865599,0.212545,0.532897,0.357892,18.96,1


### Check out this table in BigQuery Console:
- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

In [17]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=onboarding-host-5c79d5c5a45a40


---
## Review Data in BigQuery
Additional SQL queries could be used to review the data.  This section shows moving the table to a Pandas dataframe for local review in Python:

> **Note:** <p>This query only selects one column.  This means BigQuery scans less data as it does not process the other columns.  </p>

In [18]:
query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [19]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: Int64

In [20]:
df['Class'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: Float64

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [21]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [22]:
(job.ended-job.started).total_seconds()

6.173

In [23]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

70.632136 MB


Review the test/train split:

In [24]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,TEST,28485,10.00151
1,TRAIN,228097,80.08827
2,VALIDATE,28225,9.91022


Retrieve a subset of the data to a Pandas dataframe:

In [25]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [26]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,113707,1.883641,0.390920,-0.125648,4.038069,0.084600,0.282469,-0.124244,-0.013172,-0.389203,...,0.076775,0.019631,0.120873,0.134686,0.006738,-0.045759,0.0,0,b709f320-2af6-4531-b2b3-1159222dc6e9,TEST
1,40811,-0.278140,0.340027,0.978907,0.481231,0.668480,0.078513,0.557259,-0.097339,-0.336347,...,0.006569,-0.263985,-0.112467,0.635217,-0.252823,-0.247246,0.0,0,27d65f26-bb85-4575-b7b8-0c7e4c3e2a00,TEST
2,151033,0.162210,0.761509,0.022739,-0.812837,0.642273,-1.240658,1.175337,-0.315150,-0.036143,...,-0.084828,-0.004986,-0.240305,-0.150329,0.028271,0.032105,0.0,0,9923b0dc-5c9c-4215-907c-d44db20179be,TEST
3,70594,-6.873266,-6.488383,0.565778,5.293881,4.432267,-3.532376,-4.068733,-0.170392,-0.213187,...,-2.703531,0.660517,-0.726914,0.119056,1.729701,-1.992187,0.0,0,9c696429-07e2-45d8-bc02-5efe04b90eab,TEST
4,149370,2.128564,0.263455,-2.433642,0.331570,0.972006,-1.310543,0.977674,-0.564233,-0.515611,...,-0.240987,-0.211359,0.723244,0.720894,-0.122079,-0.105187,0.0,0,1ada1c57-b12c-47c7-ab96-0ec70d792584,TEST


---
## Remove Resources
see notebook "99 - Cleanup"